In [3]:
import numpy as np
import matplotlib
from scipy import signal
import utils

First of all we should upload data from npz file with the map and rod description.

In [1]:
data = np.load('data_ps1.npz')
env = data['environment']
rod = data['rod']
env,shape
rod.shape

NameError: name 'np' is not defined